# Analyzing Olympics Data with SQL and Python


In [75]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.express as px

# Import the data
olympics = pd.read_csv(r"C:\Users\yoyoj\Documents\Data\Data Camp Projects\Olympics with SQL and Python\athlete_events.csv")

# Preview the DataFrame
olympics

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [76]:
# Inspect the DataFrame
olympics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int64  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     261642 non-null  float64
 4   height  210945 non-null  float64
 5   weight  208241 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int64  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [77]:
# Check missing values
olympics.isna().sum()

id             0
name           0
sex            0
age         9474
height     60171
weight     62875
team           0
noc            0
games          0
year           0
season         0
city           0
sport          0
event          0
medal     231333
dtype: int64

The missing values in the `medal` column are because the dataset contains all competitors (not just those who won a medal). The remaining columns with missing values are not of interest to us today.

When exploring it, it looked as though some of the teams had hyphens and backslashes. Let's inspect it more closely by inspecting the unique values of the column. 

In [78]:
# Inspect the team column
olympics["team"].value_counts().to_frame()

,team
United States,17847
France,11988
Great Britain,11404
Italy,10260
Germany,9326
...,...
Briar,1
Hannover,1
Nan-2,1
Brentina,1


The `team` column is messy and sometimes contains countries separated by forward slashes or hyphens. Let's clean this by using [`.str.extract()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html) to extract the first country mentioned in the cases of slashes or hyphens (e.g., "Denmark/Sweden" becomes "Denmark").

In [79]:
# Split the team column on forward slashes and hyphens
olympics["team_clean"] = olympics["team"].str.split("[/-]").str[0]

# Preview the new column
olympics["team_clean"].unique()

array(['China', 'Denmark', 'Netherlands', 'United States', 'Finland',
       'Norway', 'Romania', 'Estonia', 'France', 'Taifun', 'Morocco',
       'Spain', 'Egypt', 'Iran', 'Bulgaria', 'Italy', 'Chad',
       'Azerbaijan', 'Sudan', 'Russia', 'Argentina', 'Cuba', 'Belarus',
       'Greece', 'Cameroon', 'Turkey', 'Chile', 'Mexico', 'Soviet Union',
       'Nicaragua', 'Hungary', 'Nigeria', 'Algeria', 'Kuwait', 'Bahrain',
       'Pakistan', 'Iraq', 'United Arab Republic', 'Lebanon', 'Qatar',
       'Malaysia', 'Germany', 'Thessalonki', 'Canada', 'Ireland',
       'Australia', 'South Africa', 'Eritrea', 'Tanzania', 'Jordan',
       'Tunisia', 'Libya', 'Belgium', 'Djibouti', 'Palestine', 'Comoros',
       'Kazakhstan', 'Brunei', 'India', 'Saudi Arabia', 'Syria',
       'Maldives', 'Ethiopia', 'United Arab Emirates', 'North Yemen',
       'Indonesia', 'Philippines', 'Singapore', 'Uzbekistan',
       'Kyrgyzstan', 'Tajikistan', 'Unified Team', 'Japan',
       'Congo (Brazzaville)', 'Switzerlan

## 🌎&nbsp;&nbsp;Bring in additional data
Let's query a MariaDB database containing information on world nations to provide some additional data. 

We will store our query result as a pandas DataFrame named `nations_data`.

In [80]:
# nations_data df generated using the following query:

q = """
    SELECT name AS country, year, population 
    FROM nations_data 
    INNER JOIN country_stats USING(country_id);
    """ 

nations_data = pd.read_csv(r"C:\Users\yoyoj\Documents\Data\Data Camp Projects\Olympics with SQL and Python\nations.data.csv")

nations_data

,country,year,population
0,Aruba,1986,62644
1,Aruba,1987,61833
2,Aruba,1988,61079
3,Aruba,1989,61032
4,Aruba,1990,62149
...,...,...,...
9509,Zimbabwe,2014,13586681
9510,Zimbabwe,2015,13814629
9511,Zimbabwe,2016,14030390
9512,Zimbabwe,2017,14236745


We now have country data that we can join with the Olympics data! We will use the [`.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method to combine the two DataFrames using the country and year columns.

A "left" join matches on rows in the olympics_data DataFrame, as some teams will not be present in the `countries_data` DataFrame.

In [81]:
# Perform a left join between the two DataFrames
olympics_full = olympics.merge(nations_data, left_on=["team_clean", "year"], right_on=["country", "year"], how="left")

# Preview our data
olympics_full

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal,team_clean,country,population
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,China,1.164970e+09
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,China,1.350695e+09
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,Netherlands,1.476009e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN,Poland,NaN,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN,Poland,Poland,3.801174e+07
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN,Poland,Poland,3.801174e+07
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN,Poland,Poland,3.866348e+07


## 🏆&nbsp;&nbsp;Which countries have the most gold medals?
Let's start by calculating and visualizing the number of gold medals won by athletes from different countries. To do this, we can use the [`.query()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method to filter for rows where the `medal` is "Gold" and then use [.group_by()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) to aggregate by our `team_clean` variable. We then use [`.count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html) to count the number of rows per team.

A second line sorts the values of our query so that we see the top teams first!

In [82]:
# Count the number of gold medals earned by a country
gold_count = olympics_full.query("medal=='Gold'").groupby("team_clean", as_index=False)["medal"].count()

# Sort the values
gold_count.sort_values(by="medal", ascending=False, inplace=True)

# Preview our count
gold_count

,team_clean,medal
194,United States,2529
175,Soviet Union,1080
74,Germany,721
93,Italy,549
75,Great Britain,545
...,...,...
37,Cicely,1
177,Suriname,1
149,Panama,1
181,Syria,1


We can visualize this using Plotly. Let's create a [choropleth map](https://plotly.com/python/choropleth-maps/) (i.e., a world map), where the country's color is based upon the medal count!

In [83]:
# Create choropleth map of gold medal counts
fig = px.choropleth(gold_count, locations="team_clean", locationmode="country names", color="medal", labels={"team_clean":"country", "medal":"Medal Count"},title="Number of Gold Medals by Country" )

fig.show()

## 📈&nbsp;&nbsp;How has the number of sports grown over time?
Another question we can ask ourselves is whether the number of individual sports has increased over time.

We will do this by grouping by the year and season (i.e., summer or winter) and then counting the number of unique using [.nunique()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html).

In [84]:
# Group by year and season and count the number of unique values
sport_count = olympics_full.groupby(["year", "season"], as_index=False)["sport"].nunique()

# Preview the DataFrame
sport_count

,year,season,sport
0,1896,Summer,9
1,1900,Summer,20
2,1904,Summer,18
3,1906,Summer,13
4,1908,Summer,24
5,1912,Summer,17
6,1920,Summer,25
7,1924,Summer,20
8,1924,Winter,10
9,1928,Summer,17


Let's plot this data using a [line plot](https://plotly.com/python/line-charts/), broken up by season!

In [85]:
# Create a line plot for Summer and Winter Olympics
fig = px.line(sport_count, x="year", y="sport", color="season", labels={"year": "Year", "sport":"Sport Count", "season":"Season"}, title="Count of Distinct Sports by Year and Season")

fig.show()

## 👪&nbsp;&nbsp;Bonus: Which countries had the highest medal count per 10 million people in 2016?
Finally, let's use the population data we took from our SQL database! One exciting way will be to reproduce a variant of [this chart](https://www.businessinsider.com/2016-rio-olympics-medals-per-capita-2016-8?r=US&IR=T) from Business Insider of the number of medals per capita.

To account for team data, we will first aggregate by the team, event, medal, and population and select the [`.first()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.first.html) medal. This will allow us to get the medals by country and event.

In [86]:
# Calculate event medals
event_medals = olympics_full.query("year == 2016")\
    .groupby(["team_clean", "event", "medal", "population"], as_index=False)["medal"].first()

# Preview the DataFrame
event_medals

,team_clean,event,population,medal
0,Algeria,"Athletics Men's 1,500 metres",40551404.0,Silver
1,Algeria,Athletics Men's 800 metres,40551404.0,Silver
2,Argentina,Hockey Men's Hockey,43590368.0,Gold
3,Argentina,Judo Women's Extra-Lightweight,43590368.0,Gold
4,Argentina,Sailing Mixed Multihull,43590368.0,Gold
...,...,...,...,...
796,Uzbekistan,"Wrestling Men's Featherweight, Greco-Roman",31847900.0,Bronze
797,Uzbekistan,"Wrestling Men's Heavyweight, Freestyle",31847900.0,Bronze
798,Uzbekistan,"Wrestling Men's Welterweight, Freestyle",31847900.0,Bronze
799,Vietnam,"Shooting Men's Air Pistol, 10 metres",93638724.0,Gold


With the event data, we can perform a similar aggregation as before and calculate the ratio of medal count to population (divided by 10 million for interpretability).

In [87]:
# Group by the team and population
medal_counts = event_medals.groupby(["team_clean", "population"], as_index=False)["medal"].count()

# Calculate the number of medals per 10000000 people
medal_counts["per_10m"] = medal_counts["medal"] / (medal_counts["population"] / 10000000)

# Sort values and take the top 20 countries
top_countries = medal_counts.sort_values(by="per_10m", ascending=False).head(20)

# Preview the DataFrame
top_countries

,team_clean,population,medal,per_10m
29,Grenada,110261.0,1,90.693899
6,Bahamas,377931.0,2,52.919713
47,New Zealand,4693200.0,18,38.353362
37,Jamaica,2906238.0,11,37.849619
19,Denmark,5728010.0,15,26.187105
16,Croatia,4174349.0,10,23.955831
58,Slovenia,2065042.0,4,19.370066
26,Georgia,3727505.0,7,18.779318
5,Azerbaijan,9757812.0,18,18.446758
30,Hungary,9814023.0,15,15.284252


We can now use a [bar chart](https://plotly.com/python/bar-charts/) to visualize the top countries by medals per capita!

In [88]:
# Create a column chart by medal per capita
fig = px.bar(top_countries, x="team_clean", y="per_10m", labels={"team_clean":"Country", "per_10m":"Medals per 10M"}, title="Number of Medals per 10 Million in Population")

fig.show()

That's all! We successfully loaded data from two different sources, performed light cleaning, and then used the data to answer three questions with interactive charts.